# DNA Challenge

## 📚 Цель проекта

Ваша задача — построить **векторные представления (эмбеддинги) ДНК-последовательностей**, и затем обучить классификаторы, чтобы достичь **максимально возможного качества на 5 биоинформатических задачах** из бенчмарка [Nucleotide Transformer Benchmark](https://huggingface.co/datasets/InstaDeepAI/nucleotide_transformer_downstream_tasks).

---

## 🧪 Датасеты и метрики

Вы будете работать с пятью подзадачами из бенчмарка. Для каждой задачи указана **основная метрика**, по которой оценивается качество модели:

---

## 🧬 Биологический смысл задач

| №  | Название задачи      | Метрика оценки | Биологическое значение |
|----|----------------------|----------------|-------------------------|
| 1  | **promoter_all**     | F1-score       | **Промотеры** — участки ДНК, с которых начинается транскрипция гена. Эти регуляторные элементы находятся перед началом гена и служат якорем для РНК-полимеразы. Задача важна для определения того, какие участки генома активно экспрессируются. |
| 2  | **enhancers**        | MCC            | **Энхансеры** — участки ДНК, усиливающие экспрессию генов. Они могут находиться далеко от самого гена и взаимодействовать с промотером через 3D-свёртку хроматина. Энхансеры играют ключевую роль в тканеспецифической экспрессии. Задача сложная из-за вариабельности энхансерных последовательностей. |
| 3  | **splice_sites_all** | Accuracy       | **Splice sites (сайты сплайсинга)** — границы между экзонами и интронами в пре-мРНК. Правильное определение этих сайтов важно для моделирования посттранскрипционных модификаций. Ошибки в сплайсинге часто приводят к заболеваниям. Задача — определить, где начинается и заканчивается сплайсинг. |
| 4  | **H3**               | MCC            | Метка **H3 (гистон H3, например, H3K4me3)** связана с **активными промотерами** и **регуляторными регионами**. Предсказание таких эпигенетических меток по ДНК-последовательности позволяет понять, какие регионы могут быть "включены" или "выключены" на уровне хроматина. |
| 5  | **H4**               | MCC            | **H4** — ещё один гистон, участвующий в **формировании нуклеосом** и упаковке ДНК. Его модификации также играют роль в регуляции транскрипции. Задача — определить участки ДНК, которые, вероятно, взаимодействуют с модифицированным H4, что даёт ключ к пониманию структуры хроматина. |

---

## 📈 Метрики качества

Ниже представлены формулы всех метрик, которые используются для оценки:

### 1. **F1-score (гармоническое среднее Precision и Recall)**

F1-score особенно важна в задачах со **сильным дисбалансом классов**.

$$
\text{F1} = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}, \quad \text{где:}
$$

$$
\text{Precision} = \frac{TP}{TP + FP}, \quad \text{Recall} = \frac{TP}{TP + FN}
$$

- **TP** — истинно положительные (True Positives)
- **FP** — ложно положительные (False Positives)
- **FN** — ложно отрицательные (False Negatives)

---

### 2. **MCC (Matthews Correlation Coefficient)**

MCC — метрика, подходящая при сильной несбалансированности классов. Принимает значение от `-1` до `1`:

$$
\text{MCC} = \frac{TP \cdot TN - FP \cdot FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
$$

- `1` — идеально
- `0` — случайное предсказание
- `-1` — полное несоответствие

---

### 3. **Accuracy (доля верно классифицированных примеров)**

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

Используется, когда классы относительно сбалансированы.

---

## 🛠️ Технические детали

### 📦 Данные

Данные загружаются из HuggingFace:

```python
from datasets import load_dataset

dataset = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks", "promoter_all")
```

Каждая задача включает поля:
- `"sequence"` — строка из символов {A, T, G, C}
- `"label"` — таргет

---

### 🧪 Базовая схема эксперимента

1. 📥 Загрузить данные
2. 🧬 Преобразовать последовательности в векторы (эмбеддинги).
3. 🤖 Обучить классификатор.
4. 📊 Посчитать метрику на test.

---

## 🏆 Задание

Постройте pipeline, который даёт **максимально высокие значения метрик на всех 5 задачах**, применяя разные подходы к построению эмбеддингов ДНК-цепочек и решению предложенных задач классификации. Итоговое сравнение решений будет происходить по среднему значению всех 5 метрик.

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
from catboost import CatBoostClassifier
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, f1_score, accuracy_score
from tqdm import tqdm
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel, AutoModelForMaskedLM
from transformers.models.bert.configuration_bert import BertConfig

import numpy as np

In [2]:
# Проверяем доступность CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Определение собственного класса Dataset для работы с последовательностями ДНК
class NTDataset(Dataset):
    def __init__(self, sequences, labels, k=6):
        self.sequences = sequences
        self.labels = labels
        self.k = k

    def __len__(self):
        return len(self.sequences)

    def _to_kmers(self, seq):
        return " ".join([seq[i:i+self.k] for i in range(len(seq) - self.k + 1)])

    # Если нужны k-меры
    # def __getitem__(self, idx):
    #     kmers = self._to_kmers(self.sequences[idx])
    #     return kmers, self.labels[idx]

    # Если нужны не k-меры, а исходная цепочка
    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        return sequence, self.labels[idx]

In [4]:
def load_nucleotide_transformer(batch_size, valid_split=-1, dataset_name='enhancers', split_state=42):
    # Загружаем тренировочный и тестовый наборы данных
    train_dataset = load_dataset(
        "InstaDeepAI/nucleotide_transformer_downstream_tasks",
        dataset_name,
        split="train",
        streaming=False,
    )

    test_dataset = load_dataset(
        "InstaDeepAI/nucleotide_transformer_downstream_tasks",
        dataset_name,
        split="test",
        streaming=False,
    )

    # Извлекаем последовательности и метки
    train_sequences = train_dataset['sequence']
    train_labels = train_dataset['label']

    # Если указано разделение на валидационный набор, разбиваем тренировочные данные
    if valid_split > 0:
        train_sequences, validation_sequences, train_labels, validation_labels = train_test_split(
            train_sequences, train_labels, test_size=valid_split, random_state=split_state
        )

    test_sequences = test_dataset['sequence']
    test_labels = test_dataset['label']

    # Создаем объекты класса NTDataset
    train_dataset = NTDataset(train_sequences, train_labels)
    test_dataset = NTDataset(test_sequences, test_labels)

    # Создаем DataLoader'ы для обучения и тестирования
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    # Создаем DataLoader для валидации, если есть разделение
    if valid_split > 0:
        validation_dataset = NTDataset(validation_sequences, validation_labels)
        valid_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
        
        print(f"Train: {len(train_loader.dataset)}, Validation: {len(valid_loader.dataset)}, Test: {len(test_loader.dataset)}")
        return train_loader, valid_loader, test_loader

    # Если нет разделения на валидацию, выводим количество образцов
    print(f"Train: {len(train_loader.dataset)}, Test: {len(test_loader.dataset)}")
    return train_loader, None, test_loader

In [5]:
def get_nt_embeddings(dataloader, tokenizer, model, pooling="mean"):
    embeddings = []
    labels = []

    model.to(device)
    model.eval()

    for batch in tqdm(dataloader, desc="Generating embeddings"):
        sequences, lbls = batch

        tokens = tokenizer(
            list(sequences), padding=True, truncation=True, max_length=512, return_tensors="pt"
        )
        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)

        with torch.no_grad():
            with autocast(enabled=torch.cuda.is_available()):
                outputs = model(
                    input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True
                )

            last_hidden = outputs.hidden_states[-1]

            if pooling == "mean":
                attention_mask_exp = attention_mask.unsqueeze(-1)
                sum_embeddings = torch.sum(last_hidden * attention_mask_exp, dim=1)
                sum_mask = attention_mask_exp.sum(dim=1).clamp(min=1e-9)
                pooled = sum_embeddings / sum_mask
            elif pooling == "max":
                pooled = last_hidden.masked_fill(attention_mask.unsqueeze(-1) == 0, -1e9)
                pooled = torch.max(pooled, dim=1).values
            else:
                raise ValueError("pooling must be 'mean' or 'max'")

        embeddings.append(pooled.cpu().numpy())
        labels.append(lbls.numpy())

    return np.vstack(embeddings), np.concatenate(labels)

In [6]:
def classify_with_dnabert(dataset_name, metric, embedding_fn = get_nt_embeddings):
    # Поскольку в CatBoost нет Early Stopping по метрике MCC, используется в данном случае F1
    eval_metric = "F1" if metric == "MCC" else metric
        
    batch_size = 128 # Размер батча можно сделать поменьше при необходимости
    train_loader, valid_loader, test_loader = load_nucleotide_transformer(batch_size, valid_split=0.1, dataset_name=dataset_name)

    model_name = "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)

    model.eval()

    X_train, y_train = embedding_fn(train_loader, tokenizer, model)
    X_valid, y_valid = embedding_fn(valid_loader, tokenizer, model)
    X_test, y_test = embedding_fn(test_loader, tokenizer, model)

    clf = CatBoostClassifier(
        iterations=3_000,
        learning_rate=0.02,
        depth=4,
        task_type="GPU",
        eval_metric=eval_metric,
        early_stopping_rounds=100,
        use_best_model=True,
        verbose=50
    )

    clf.fit(X_train, y_train, eval_set=(X_valid, y_valid) if valid_loader else None)

    y_pred = clf.predict(X_test)

    if metric == "F1":
        metric = f1_score(y_test, y_pred, average="binary")
        print(f"F1 Score: {metric:.4f}")
    elif metric == "Accuracy":
        metric = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {metric:.4f}")
    else:
        metric = matthews_corrcoef(y_test, y_pred)
        print(f"MCC: {metric:.4f}")

    return metric

In [7]:
# Список датасетов, на которых будет измеряться метрика во время DNAChallenge
# В оригинальном бенчмарке 18 датасетов, для простоты решения задачи ограничимся пятью

DATASETS = [
    ("promoter_all", "F1"),
    ("enhancers", "MCC"),
    ("splice_sites_all", "Accuracy"),
    ("H3", "MCC"),
    ("H4", "MCC")
]

result_metrics = {}

for dataset, metric in DATASETS:
    result_metrics[dataset] = classify_with_dnabert(dataset, metric)

Train: 47948, Validation: 5328, Test: 5920


/tmp/ipykernel_2126299/3408497357.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):
Generating embeddings: 100%|████████████████████| 47/47 [00:07<00:00,  6.69it/s]


0:	learn: 0.8159836	test: 0.8084853	best: 0.8084853 (0)	total: 48.4ms	remaining: 2m 25s
50:	learn: 0.8763783	test: 0.8700432	best: 0.8700432 (50)	total: 1.35s	remaining: 1m 17s
100:	learn: 0.8944375	test: 0.8909730	best: 0.8909730 (100)	total: 2.64s	remaining: 1m 15s
150:	learn: 0.9046207	test: 0.8999024	best: 0.8999415 (148)	total: 3.92s	remaining: 1m 13s
200:	learn: 0.9102949	test: 0.9060194	best: 0.9060194 (200)	total: 5.21s	remaining: 1m 12s
250:	learn: 0.9147380	test: 0.9097963	best: 0.9106069 (249)	total: 6.51s	remaining: 1m 11s
300:	learn: 0.9179320	test: 0.9121857	best: 0.9124710 (276)	total: 7.8s	remaining: 1m 9s
350:	learn: 0.9204676	test: 0.9159420	best: 0.9159420 (350)	total: 9.1s	remaining: 1m 8s
400:	learn: 0.9229491	test: 0.9175198	best: 0.9179061 (381)	total: 10.4s	remaining: 1m 7s
450:	learn: 0.9250378	test: 0.9181467	best: 0.9181467 (449)	total: 11.7s	remaining: 1m 5s
500:	learn: 0.9262370	test: 0.9197840	best: 0.9199923 (499)	total: 12.9s	remaining: 1m 4s
550:	learn:

/tmp/ipykernel_2126299/3408497357.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):
Generating embeddings: 100%|██████████████████████| 4/4 [00:00<00:00, 11.24it/s]


0:	learn: 0.8008929	test: 0.8193717	best: 0.8193717 (0)	total: 23.4ms	remaining: 1m 10s
50:	learn: 0.8423627	test: 0.8579053	best: 0.8588549 (47)	total: 1.12s	remaining: 1m 4s
100:	learn: 0.8607220	test: 0.8709030	best: 0.8714859 (98)	total: 2.22s	remaining: 1m 3s
150:	learn: 0.8711902	test: 0.8777555	best: 0.8789298 (149)	total: 3.31s	remaining: 1m 2s
200:	learn: 0.8780596	test: 0.8908123	best: 0.8908123 (200)	total: 4.39s	remaining: 1m 1s
250:	learn: 0.8822788	test: 0.8956811	best: 0.8970100 (241)	total: 5.46s	remaining: 59.8s
300:	learn: 0.8857715	test: 0.8977424	best: 0.8990704 (262)	total: 6.55s	remaining: 58.7s
350:	learn: 0.8887076	test: 0.8995343	best: 0.9002660 (349)	total: 7.68s	remaining: 58s
400:	learn: 0.8914430	test: 0.9037823	best: 0.9037823 (391)	total: 8.77s	remaining: 56.8s
450:	learn: 0.8937620	test: 0.9070385	best: 0.9070385 (439)	total: 9.84s	remaining: 55.6s
500:	learn: 0.8964955	test: 0.9065606	best: 0.9084881 (462)	total: 10.9s	remaining: 54.4s
550:	learn: 0.898

...

0:	learn: 0.4195062	test: 0.4107407	best: 0.4107407 (0)	total: 22.7ms	remaining: 1m 8s
50:	learn: 0.4791770	test: 0.4707407	best: 0.4733333 (31)	total: 342ms	remaining: 19.8s
100:	learn: 0.4951852	test: 0.4862963	best: 0.4862963 (100)	total: 669ms	remaining: 19.2s
150:	learn: 0.5077778	test: 0.4896296	best: 0.4922222 (146)	total: 1.01s	remaining: 19s
200:	learn: 0.5169959	test: 0.4959259	best: 0.4962963 (195)	total: 1.34s	remaining: 18.7s
250:	learn: 0.5242387	test: 0.5062963	best: 0.5077778 (248)	total: 1.68s	remaining: 18.3s
300:	learn: 0.5325514	test: 0.5148148	best: 0.5162963 (288)	total: 2s	remaining: 18s
350:	learn: 0.5396296	test: 0.5207407	best: 0.5214815 (344)	total: 2.33s	remaining: 17.6s
400:	learn: 0.5459259	test: 0.5222222	best: 0.5233333 (371)	total: 2.64s	remaining: 17.1s
450:	learn: 0.5506584	test: 0.5248148	best: 0.5251852 (445)	total: 2.96s	remaining: 16.7s
500:	learn: 0.5556790	test: 0.5325926	best: 0.5333333 (498)	total: 3.29s	remaining: 16.4s
550:	learn: 0.5595062	

/tmp/ipykernel_2126299/3408497357.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):
Generating embeddings: 100%|████████████████████| 12/12 [00:03<00:00,  3.67it/s]


0:	learn: 0.7715030	test: 0.7520107	best: 0.7520107 (0)	total: 21.1ms	remaining: 1m 3s
50:	learn: 0.8016226	test: 0.7896254	best: 0.7925341 (40)	total: 1s	remaining: 57.9s
100:	learn: 0.8133312	test: 0.8023256	best: 0.8049311 (97)	total: 1.98s	remaining: 56.8s
150:	learn: 0.8210312	test: 0.8150834	best: 0.8156749 (139)	total: 2.95s	remaining: 55.7s
200:	learn: 0.8278463	test: 0.8215586	best: 0.8224163 (183)	total: 3.92s	remaining: 54.7s
250:	learn: 0.8320214	test: 0.8230994	best: 0.8245614 (229)	total: 4.89s	remaining: 53.6s
300:	learn: 0.8352756	test: 0.8245614	best: 0.8256747 (279)	total: 5.87s	remaining: 52.6s
350:	learn: 0.8385622	test: 0.8263736	best: 0.8289474 (345)	total: 6.83s	remaining: 51.6s
400:	learn: 0.8421053	test: 0.8313783	best: 0.8319883 (399)	total: 7.8s	remaining: 50.6s
450:	learn: 0.8453152	test: 0.8338235	best: 0.8340675 (424)	total: 8.78s	remaining: 49.6s
500:	learn: 0.8479190	test: 0.8359088	best: 0.8367647 (478)	total: 9.76s	remaining: 48.7s
550:	learn: 0.849996

/tmp/ipykernel_2126299/3408497357.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=torch.cuda.is_available()):
Generating embeddings: 100%|████████████████████| 12/12 [00:03<00:00,  3.74it/s]


0:	learn: 0.7262125	test: 0.6981132	best: 0.6981132 (0)	total: 20.7ms	remaining: 1m 2s
50:	learn: 0.8088421	test: 0.7905282	best: 0.7905282 (50)	total: 1s	remaining: 58s
100:	learn: 0.8246464	test: 0.8086643	best: 0.8093948 (98)	total: 2s	remaining: 57.3s
150:	learn: 0.8354088	test: 0.8125561	best: 0.8140162 (138)	total: 2.97s	remaining: 56s
200:	learn: 0.8412958	test: 0.8160714	best: 0.8175313 (178)	total: 3.96s	remaining: 55.1s
250:	learn: 0.8461169	test: 0.8228980	best: 0.8232143 (243)	total: 4.95s	remaining: 54.2s
300:	learn: 0.8474609	test: 0.8254252	best: 0.8254252 (296)	total: 5.94s	remaining: 53.2s
350:	learn: 0.8495355	test: 0.8251121	best: 0.8254252 (296)	total: 6.93s	remaining: 52.3s
400:	learn: 0.8521856	test: 0.8254252	best: 0.8261649 (385)	total: 7.92s	remaining: 51.3s
450:	learn: 0.8555279	test: 0.8272158	best: 0.8272158 (432)	total: 8.91s	remaining: 50.3s
500:	learn: 0.8584545	test: 0.8282648	best: 0.8290063 (484)	total: 9.89s	remaining: 49.4s
550:	learn: 0.8618975	test

In [8]:
# Максимальная ширина для выравнивания
max_name_len = max(len(name) for name in result_metrics.keys())
line_format = f"{{:<{max_name_len}}} : {{:.4f}}"

# Выводим таблицу
print("Metrics per dataset:\n")
for dataset, score in result_metrics.items():
    print(line_format.format(dataset, score))

# Среднее значение
mean_score = sum(result_metrics.values()) / len(result_metrics)
print("\n" + "-" * (max_name_len + 12))
print(f"{'Average'.ljust(max_name_len)} : {mean_score:.4f}")

Metrics per dataset:

promoter_all     : 0.9186
enhancers        : 0.4983
splice_sites_all : 0.5447
H3               : 0.6938
H4               : 0.7237

----------------------------
Average          : 0.6758
